## 1: Doing a basic completion with GPT3

Here we'll load in our API keys, organizations, and see if we can figure out what one plus one equals.

Start with the imports

In [1]:
import os, openai

Now we need to set the API key and (if applicable) the organization you belong to if that's set up. You can manually put them in or bring them in from the environement.

In [2]:
openai.api_key = os.environ['OPENAI_KEY']

In [3]:
try:
    openai.organization = os.environ['OPENAI_ORG']
    print('Organization Added')
except:
    print('No Organization')

Organization Added


*Run this if you want to charge your personal instead of org*

In [4]:
#openai.organization=None

You can do a query via the Completion API endpoint. Let's create a simple completion prompt.

The three quotes are used for multiline strings in python.

In [6]:
prompt = """Question: What is 1 + 1?
Answer:"""

In [7]:
print(prompt)

Question: What is 1 + 1?
Answer:


Note: We never want to end a prompt with a blank space. We can see why in a minute.

Let's go ahead and query it on the smallest model, ADA

In [8]:
response = openai.Completion.create(prompt=prompt, engine="ada")

This gives us a JSON of the responses.

In [9]:
print(response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " 1.100.Don't You Know That by Boston Pops?You'll"
    }
  ],
  "created": 1623711797,
  "id": "cmpl-3ApttKP6xQmxVdvc9c6WZqDhZ0Weq",
  "model": "ada:2020-05-03",
  "object": "text_completion"
}


Depending on how many completions we asked for (variations) the choices array can be larger.

In this case, there's just 1 option. Let's print it.

In [11]:
print(response['choices'][0]['text'])

 1.100.Don't You Know That by Boston Pops?You'll


We can see the options that are available by requesting the possible completions using the `logprobs` parameters

In [12]:
response = openai.Completion.create(prompt=prompt, engine="ada", logprobs=5)

*note: logprobs are the log of the probability that that token comes next; at 0 temperature the most likely logprob (least negative) is chosen. As temperature goes up, more unlikely tokens can be selected from the distribution

In [13]:
print(response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": {
        "text_offset": [
          32,
          34,
          37,
          41,
          46,
          50,
          51,
          55,
          58,
          60,
          66,
          73,
          74,
          75,
          79,
          80
        ],
        "token_logprobs": [
          -1.1586264,
          -2.0373125,
          -0.91038746,
          -5.251942,
          -13.529189,
          -3.0176218,
          -2.6854093,
          -0.39519098,
          -3.6330554,
          -5.4778113,
          -6.801372,
          -2.9611325,
          -4.790423,
          -6.553976,
          -0.5858207,
          -3.5208912
        ],
        "tokens": [
          " 1",
          " is",
          " the",
          " root",
          " pen",
          "-",
          "name",
          " of",
          " a",
          " small",
          " branch",
          ",",
          "\n",
          "Th

That's a lot of text. Let's just grab the most likely first token.

In [19]:
print(response['choices'][0]['logprobs']['top_logprobs'][0])

{
  "\n": -3.3884075,
  " 1": -1.1586264,
  " A": -3.1307986,
  " It": -3.1697712,
  " The": -2.7225835
}


Hmm, let's try a smarter engine.

In [22]:
response = openai.Completion.create(prompt=prompt, engine="babbage", logprobs=5)

In [24]:
print(response['choices'][0]['text'])

 21.
(1 + 1) (1 + 1) = 21.


In [25]:
print(response['choices'][0]['logprobs']['top_logprobs'][0])

{
  " 1": -1.3638556,
  " 2": -1.9325984,
  " 3": -2.5479558,
  " 4": -2.9448903,
  " 5": -3.400267
}


In [26]:
response = openai.Completion.create(prompt=prompt, engine="babbage", logprobs=5, temperature=0)

Ah, so it was trying to write out that 1+1=2 and then it was just getting confused!

In [27]:
print(response['choices'][0]['text'])

 1 + 1 = 2

Q: What is the sum of the first


Maybe `ada` could have done it?

In [28]:
response = openai.Completion.create(prompt=prompt, engine="ada", logprobs=5, temperature=0)
print(response['choices'][0]['text'])

 1 + 1 = 1.

This is a question that is often asked


Nope it couldn't have. Well, let's see how curie does.

In [29]:
response = openai.Completion.create(prompt=prompt, engine="curie", temperature=0)
print(response['choices'][0]['text'])

 2
Question: What is 2 + 2?
Answer: 4
Question


Well, curie understands what we're asking, but it keeps going. Why's it do that?

Well, there's a parameter called 'stop' that can take up to 4 sequences for when to stop generating.

In [30]:
response = openai.Completion.create(prompt=prompt, engine="curie", temperature=0, stop=['\n'])
print(response['choices'][0]['text'])

 2
